In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain.document_loaders import PyPDFLoader

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("ai_industry.pdf")

# 페이지 별 문서 로드
docs = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

import faiss
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model

In [6]:
while True: 
    print("========================")
    query = input("질문을 입력하세요: ")
    if query == "exit":
        break
    response = rag_chain_debug.invoke(query)

    print("Final Response:")
    print(response.content)

질문을 입력하세요:  유로폴의 법집행 보고서를 요약해줘


Debug Output: 유로폴의 법집행 보고서를 요약해줘
Debug Output: {'context': [Document(metadata={'source': 'ai_industry.pdf', 'page': 5}, page_content='유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서 발간n유로폴의 보고서에 따르면 AI는 고급 데이터 분석, 디지털 증거 수집, 이미지와 비디오 분석 등에 활용되어 법 집행'), Document(metadata={'source': 'ai_industry.pdf', 'page': 5}, page_content='£유로폴, 법 집행에서 AI 기술의 윤리적이고 투명한 구현을 위한 고려사항 제시nEU 사법기관 유로폴(Europol)이 2024년 9월 24일 법 집행에서 효과적 범죄 퇴치를'), Document(metadata={'source': 'ai_industry.pdf', 'page': 1}, page_content='예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표·····························2   ▹ 유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서')], 'question': '유로폴의 법집행 보고서를 요약해줘'}
Final Response:
유로폴은 법 집행에서 AI의 이점과 과제를 다룬 보고서를 발표하였으며, AI가 고급 데이터 분석, 디지털 증거 수집, 이미지 및 비디오 분석 등 다양한 분야에서 활용될 수 있음을 강조했습니다. 보고서는 또한 AI 기술의 윤리적이고 투명한 구현을 위한 고려사항을 제시하고, 법 집행에서 효과적인 범죄 퇴치를 위한 방안을 모색하고 있습니다.


질문을 입력하세요:  oecd가 무슨일을 했지


Debug Output: oecd가 무슨일을 했지
Debug Output: {'context': [Document(metadata={'source': 'ai_industry.pdf', 'page': 6}, page_content='발간∙OECD는 G7 회원국이 작성한 설문 응답 및 OECD와 UNESCO의 연구를 토대로 공공 부문에서 AI 활용 모범사례와 거버넌스 프레임워크, 정책 옵션과 관련된 종합적'), Document(metadata={'source': 'ai_industry.pdf', 'page': 1}, page_content='▹ OECD, 공공 부문의 AI 도입을 위한 G7 툴킷 발표··························································4   ▹'), Document(metadata={'source': 'ai_industry.pdf', 'page': 6}, page_content='£OECD, G7의 사례를 토대로 공공 부문의 AI 도입을 안내하는 지침 마련nOECD가 2024년 10월 15일 안전하고 신뢰할 수 있는 AI의 원칙을 실행 가능한 정책으로')], 'question': 'oecd가 무슨일을 했지'}
Final Response:
OECD는 G7 회원국의 설문 응답 및 연구를 바탕으로 공공 부문에서 AI 활용 모범사례와 거버넌스 프레임워크, 정책 옵션에 대한 종합적인 가이드라인을 마련하고, 2024년 10월 15일에 안전하고 신뢰할 수 있는 AI의 원칙을 실행 가능한 정책으로 발표했습니다.


질문을 입력하세요:  cb인사이츠는 스타트업에 대해 어떻게 생각하는지 알려줘


...

질문을 입력하세요:  ai전문가들과 일반적인 기술 전문가의 차이는?


Debug Output: ai전문가들과 일반적인 기술 전문가의 차이는?
Debug Output: {'context': [Document(metadata={'source': 'ai_industry.pdf', 'page': 20}, page_content='AI 기술 전문가는 일반적인 기술 전문가 대비 기술 산업의 미래와 자기 경력에 대하여 낙관적∙이번 조사는 520명의 미국 정규직 기술 전문가와 390명의 인사 전문가의 응답을'), Document(metadata={'source': 'ai_industry.pdf', 'page': 20}, page_content='£AI 전문가들, 일반적인 기술 전문가보다 직업 전망에 낙관적n미국 기술직 채용 플랫폼 다이스(Dice)의 조사에 따르면, AI 기술 전문가는 일반적인 기술 전문가 대비 기술'), Document(metadata={'source': 'ai_industry.pdf', 'page': 20}, page_content='전문가들은 여타 기술 전문가 대비 AI 도구 사용에도 적극적이며, 업무에 생성AI가 상당한 영향을 미친다는 응답도 36%로 여타 기술 전문가(22%) 대비 높은 수치를 기록')], 'question': 'ai전문가들과 일반적인 기술 전문가의 차이는?'}
Final Response:
AI 전문가들은 일반적인 기술 전문가보다 직업 전망에 더 낙관적이며, AI 도구 사용에 더 적극적이고, 생성 AI가 업무에 미치는 영향에 대한 인식이 더 높습니다.


질문을 입력하세요:  가트너가 뭐야?


Debug Output: 가트너가 뭐야?
Debug Output: {'context': [Document(metadata={'source': 'ai_industry.pdf', 'page': 21}, page_content='가트너 예측, AI로 인해 엔지니어링 인력의 80%가 역량 향상 필요n가트너에 따르면 생성AI의 도입으로 중장기적으로 소프트웨어 엔지니어링에서 데이터 과학 및 AI/ML 역량의'), Document(metadata={'source': 'ai_industry.pdf', 'page': 1}, page_content='▹ 가트너 예측, AI로 인해 엔지니어링 인력의 80%가 역량 향상 필요 ·····························19   ▹ 인디드 조사 결과, 생성AI가 인간'), Document(metadata={'source': 'ai_industry.pdf', 'page': 1}, page_content='▹ 다이스 조사, AI 전문가의 73%는 2025년 중 이직 고려················································18   ▹ 가트너')], 'question': '가트너가 뭐야?'}
Final Response:
가트너는 정보 기술, 소프트웨어, 서비스 및 관련 분야에 대한 연구 및 자문 서비스를 제공하는 글로벌 리서치 회사입니다.


질문을 입력하세요:  exit


# 문제점
1. cb인사이츠는 스타트업이 실패할 것으로 예상함 ->챗봇대답: 긍정적으로 평가하고 있다
2. 카나나는 나다운 ai 또는 카(카오의 사명)+na(tive)+na(tural) ->챗봇대답: natural
3. 노벨상 수상자: 물리학상 수상자와 화학상 수상자가 있음 ->챗봇대답: 물리학상 수상자만 언급
4. gaira는 미국무부가 2024년 9월 23일 발표 ->챗봇대답: 미국무부가 23년 10월 30일 발표
5. Ori Goshen은 기업이 아니라 이스라엘 스타트업 AI12의 ceo ->챗봇대답: 오리고센은 미국기업입니다.
6. 가트너는 시장조사기관 ->챗봇대답: 글로벌 리서치 회사입니다 ?? hallucination??

# 개선방안
* 청크사이즈 늘리기
* 프롬프트엔지니어링